In [1]:
import sys
sys.path.append('..')
from utils.paths import get_file_path

import polars as pl
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
def estilo(fig):
    fig.update_layout(
        title_font=dict(size=24, family="Arial", color="black"),
        xaxis_tickangle=-45,
        xaxis_tickfont=dict(size=12),
        yaxis_tickfont=dict(size=12),
        plot_bgcolor="white",
        paper_bgcolor="white",
        showlegend=False,
        margin=dict(t=80, l=40, r=20, b=120),
    )
    return fig

In [3]:
df = pl.scan_csv(
    get_file_path('cleaned', 'exportaciones'),
    low_memory=True
).collect()

### Sofisticación exportadora - Ratio AGRENA/FOBDOL por producto

#### __¿Qué es el ratio_ag_fob?__

El ratio_ag_fob es el cociente entre el valor agregado nacional (AGRENA) y el valor total de exportación en dólares FOB (FOBDOL) para cada producto. Se interpreta como una medida de sofisticación exportadora:

+ __Ratio alto (cercano a 1.0):__ Indica que casi todo el valor exportado proviene de insumos o procesos nacionales. Es decir, Colombia está generando valor internamente antes de exportar, lo que implica mayor desarrollo productivo.

+ __Ratio bajo (cercano a 0.0):__ Sugiere que gran parte del producto exportado no tiene aporte nacional significativo, sino que podría tratarse de reexportaciones, maquila o ensamblajes sin insumos locales.

+ __Ratio = 0.0:__ Significa que no hubo ningún valor agregado nacional. Aunque el producto se exportó, Colombia no aportó insumos ni transformación local, y por tanto no generó sofisticación económica interna.

### Productos con alta sofisticación exportadora

In [37]:
productos_sofisticados = (
    df.filter((pl.col("FOBDOL") > 50_000) & (pl.col("AGRENA") > 0))
    .with_columns((pl.col("AGRENA") / pl.col("FOBDOL")).alias("ratio_ag_fob"))
    .sort("ratio_ag_fob", descending=True)
)
productos_sofisticados = productos_sofisticados[['POSARA','AGRENA', 'FOBDOL', 'ratio_ag_fob']]

In [38]:
print("Productos con alta sofisticación (AGRENA > 0):")
productos_sofisticados

Productos con alta sofisticación (AGRENA > 0):


POSARA,AGRENA,FOBDOL,ratio_ag_fob
i64,f64,f64,f64
8504230000,2.2493e6,352000.0,6.390137
603199000,1.4422e6,824390.33,1.749358
603199000,985372.47,836207.82,1.178382
603110000,2.7781e6,2.7498e6,1.01031
803901100,130768.61,130765.3,1.000025
…,…,…,…
3903900000,0.06,206075.86,2.9115e-7
3903190000,0.02,78030.02,2.5631e-7
8537109000,0.05,217783.6,2.2959e-7


>###### __Finalidad:__ Identificar aquellos productos que generan valor en Colombia antes de ser exportados, reflejando una economía más compleja, con mayor contenido local y desarrollo productivo interno.

### Top 10 productos

In [55]:
top_agrena = (
    df.filter(pl.col("AGRENA") > 0)
      .group_by("POSARA")
      .agg(pl.col("FOBDOL").sum().alias("total_fob"))
      .sort("total_fob", descending=True)
      .head(10)
)

prod_agrena = {
    2701120010: 'Hulla bituminosa, incluso pulverizada, sin aglomerar',
    7202600000: 'Ferroníquel',
    803901100:  'Bananas y plátanos frescos (Cavendish y similares)',
    603199090:  'Flores y capullos cortados frescos (otras variedades)',
    603110000:  'Plantas ornamentales vivas',
    3902100000: 'Polietileno en formas primarias (<0.94 densidad)',
    3904102000: 'Polipropileno en formas primarias',
    803001200:  'Otras bananas y plátanos frescos',
    901119000:  'Café sin tostar, sin descafeinar',
    1701999000: 'Otros azúcares crudos y mezclas de azúcar'
}

In [56]:
top_agrena = top_agrena.to_pandas()
top_agrena['POSARA'] = top_agrena['POSARA'].map(prod_agrena)
top_agrena = top_agrena.rename(columns={'POSARA': 'PRODUCTOS'})

top_agrena

,PRODUCTOS,total_fob
0,"Hulla bituminosa, incluso pulverizada, sin agl...",8.858509e+10
1,Ferroníquel,1.496337e+10
2,Bananas y plátanos frescos (Cavendish y simila...,1.076880e+10
3,Flores y capullos cortados frescos (otras vari...,5.725307e+09
4,Plantas ornamentales vivas,5.020633e+09
5,Polietileno en formas primarias (<0.94 densidad),4.828464e+09
6,Polipropileno en formas primarias,4.634667e+09
7,Otras bananas y plátanos frescos,4.587053e+09
8,"Café sin tostar, sin descafeinar",3.973758e+09
9,Otros azúcares crudos y mezclas de azúcar,3.805083e+09


#### Conclusiones: 

+ Se identificaron 572,801 registros con valor agregado positivo.

+ Estos productos reflejan una base productiva más fuerte en el país.

+ Una alta proporción de exportaciones tiene algún grado de aporte nacional, lo cual es positivo para la sostenibilidad económica y el empleo local.

+ Es clave fomentar e incentivar estos productos, pues generan más beneficios para el aparato productivo nacional.

### Productos con alto valor exportado pero sin aporte nacional (AGRENA = 0)

In [39]:
productos_sin_agrena = (
    df.filter((pl.col("FOBDOL") > 50_000) & (pl.col("AGRENA") == 0))
    .with_columns(pl.lit(0.0).alias("ratio_ag_fob"))
    .sort("FOBDOL", descending=True)
)
productos_sin_agrena = productos_sin_agrena[['POSARA', 'AGRENA', 'FOBDOL', 'ratio_ag_fob']]

In [40]:
print("Productos con alto valor FOB pero sin aporte nacional (AGRENA = 0):")
productos_sin_agrena

Productos con alto valor FOB pero sin aporte nacional (AGRENA = 0):


POSARA,AGRENA,FOBDOL,ratio_ag_fob
i64,f64,f64,f64
2709000000,0.0,1.1251e9,0.0
2709000000,0.0,1.0436e9,0.0
2709000000,0.0,9.65602149e8,0.0
2709000000,0.0,9.17236306e8,0.0
2709000000,0.0,8.66289295e8,0.0
…,…,…,…
8418991000,0.0,50000.24,0.0
603129000,0.0,50000.2,0.0
1701999000,0.0,50000.2,0.0


>###### __Finalidad:__ Visibilizar exportaciones donde Colombia no interviene significativamente en la generación de valor, lo que puede implicar reexportaciones o actividades con bajo impacto interno.

### Top 10 productos

In [57]:
top_sin_agrena = (
    df.filter(pl.col("AGRENA") == 0)
      .group_by("POSARA")
      .agg(pl.col("FOBDOL").sum().alias("total_fob"))
      .sort("total_fob", descending=True)
      .head(10)
)

prod_sin_agrena = {
    2709000000: 'Petróleos crudos',
    901119000:  'Café sin tostar, sin descafeinar',
    7108120000: 'Gemas y metales preciosos',
    2710192200: 'Fuelóleos y gasóleos pesados',
    2701120010: 'Hulla bituminosa, incluso pulverizada, sin aglomerar',
    2704001000: 'Coques y semicoques de hulla, carbón de retorta',
    2710192100: 'Preparaciones de aceites minerales (≥70 %)',
    2710129200: 'Gasóleo (diésel) marino',
    1511100000: 'Aceite de palma crudo',
    2701120090: 'Otras hullas bituminosas (resto)'
}

In [58]:
top_sin_agrena = top_sin_agrena.to_pandas()
top_sin_agrena['POSARA'] = top_sin_agrena['POSARA'].map(prod_sin_agrena)
top_sin_agrena = top_sin_agrena.rename(columns={'POSARA': 'PRODUCTOS'})

top_sin_agrena

,PRODUCTOS,total_fob
0,Petróleos crudos,2.699387e+11
1,"Café sin tostar, sin descafeinar",4.234912e+10
2,Gemas y metales preciosos,3.683101e+10
3,Fuelóleos y gasóleos pesados,2.402246e+10
4,"Hulla bituminosa, incluso pulverizada, sin agl...",2.012835e+10
5,"Coques y semicoques de hulla, carbón de retorta",1.198304e+10
6,Preparaciones de aceites minerales (≥70 %),1.121156e+10
7,Gasóleo (diésel) marino,4.768040e+09
8,Aceite de palma crudo,4.236823e+09
9,Otras hullas bituminosas (resto),3.637415e+09


#### Conclusiones:

+ Se registraron 693,428 productos exportados sin aporte nacional.

+ Aunque generan cifras altas de exportación en dólares, no promueven desarrollo interno ni sofisticación.

+ Esto puede indicar dependencia de flujos logísticos (por ejemplo, zonas francas o productos ensamblados con insumos extranjeros).

+ Es necesario revisar políticas industriales o de encadenamientos para que estas exportaciones generen más valor local.

In [43]:
data = pd.DataFrame({
    "Tipo de producto": [
        "Con valor agregado nacional (AGRENA > 0)",
        "Sin valor agregado nacional (AGRENA = 0)"
    ],
    "Cantidad de productos": [572_801, 693_428]
})


fig = px.bar(
    data,
    x="Tipo de producto",
    y="Cantidad de productos",
    text="Cantidad de productos",
    color="Tipo de producto",
    title="Comparación de productos exportados según valor agregado nacional"
)
fig.update_layout(
    yaxis_title="Número de registros",
    xaxis_title="",
    title_font=dict(size=24, family="Arial", color="black"),
    xaxis_tickfont=dict(size=12),
    yaxis_tickfont=dict(size=12),
    plot_bgcolor="white",
    paper_bgcolor="white",
    showlegend=False
)

fig.show()

### Conclusiones finales:

Hay más productos exportados sin aporte nacional directo (AGRENA = 0 → 693,428 filas), lo cual sugiere un volumen alto de reexportación o productos de bajo procesamiento local.

En contraste, solo 572,801 productos tienen algún grado de valor agregado nacional (AGRENA > 0), lo que indica espacio para mejorar la sofisticación de la oferta exportable.

### Precios unitarios - FOBDOL/CANTIDA por producto

In [13]:
precios_unit = (df.group_by("POSARA").agg([
    pl.col("FOBDOL").sum(),
    pl.col("CANTIDAD").sum()
]).with_columns(
    (pl.col("FOBDOL") / pl.col("CANTIDAD")).alias("precio_unitario")
).sort("precio_unitario", descending=True))

In [15]:
precios_unit

POSARA,FOBDOL,CANTIDAD,precio_unitario
i64,f64,f64,f64
4811519090,35097.1,0.0,inf
3906909090,6.7950e6,0.0,inf
3004201090,1.2325e7,0.0,inf
703200000,26791.5,0.0,inf
4823903000,1.2288e6,0.0,inf
…,…,…,…
3824820000,1071.06,971287.84,0.001103
5208220000,5.6135e6,5.2119e9,0.001077
8305200000,2.7731e7,5.9994e10,0.000462


### Concentración de productos - Índice Herfindahl

In [16]:
fob_productos = df.group_by("POSARA").agg(pl.col("FOBDOL").sum())
total_fob_prod = fob_productos.select(pl.col("FOBDOL").sum()).item()

hhi_productos = (fob_productos.with_columns(
    (pl.col("FOBDOL") / total_fob_prod).alias("participacion")
).with_columns(
    (pl.col("participacion") ** 2).alias("participacion_cuadrada")
).select(pl.col("participacion_cuadrada").sum())).item()

In [17]:
hhi_productos

0.12490095172142265

### Productos de alto valor agregado (AGRENA/FOBDOL > 0.7)

In [59]:
sofisticacion = (df.group_by("POSARA").agg([
    pl.col("AGRENA").sum(),
    pl.col("FOBDOL").sum()
]).with_columns(
    (pl.col("AGRENA") / pl.col("FOBDOL")).alias("ratio_ag_fob")
).sort("ratio_ag_fob", descending=True))

alto_valor = sofisticacion.filter(pl.col("ratio_ag_fob") > 0.7)

In [60]:
alto_valor

POSARA,AGRENA,FOBDOL,ratio_ag_fob
i64,f64,f64,f64
4802550010,68055.51,68397.5,0.995
2941104000,74561.11,75032.26,0.993721
2918153000,1.1389e8,1.1699e8,0.973513
2918140000,5.7918e8,5.9627e8,0.971337
4011100000,9.9407e6,1.0277e7,0.967304
…,…,…,…
7002320000,144603.99,204739.65,0.706282
5514303000,491319.09,697115.87,0.704788
2106902090,4.9446e6,7.0215e6,0.704217
